### Component Of RAG Application

#### Create a simple Rag Application

In [ ]:
!pip install -U langchain-chroma
!pip install -U DeepEval

In [ ]:
from langchain_ollama import ChatOllama
from deepeval.tracing import observe, update_current_span
import os
os.environ["CONFIDENT_TRACE_FLUSH"] = "YES"

@observe(type="llm", model="llama3.2:latest")
def get_local_llm(model_name):
    return ChatOllama(
        base_url="http://localhost:11434",
        model=model_name,
        temperature=0.5,
        max_tokens=500
)

In [ ]:
# Load data from Web
from langchain_community.document_loaders import SeleniumURLLoader
loader = SeleniumURLLoader(urls=["https://www.descope.com/learn/post/mcp"])
data = loader.load()
# Convert Data Into Chunks
from langchain_text_splitters import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000,
                                               chunk_overlap=150,
                                               separators=["\n\n", "\n", ". ", " ", ""])
docs = text_splitter.split_documents(data)
# Create Embedding and Vector Store
from langchain_ollama import OllamaEmbeddings
from langchain_chroma import Chroma
# Note: Ensure Ollama is running and has the model available
ollama_embedding = OllamaEmbeddings(model="llama3.2:latest")
vector_store = Chroma.from_documents(documents=docs, 
                                     embedding= ollama_embedding)
## Setting up retrieval augmented generation (RAG)
retriever = vector_store.as_retriever()
## Initialize LLM For Generation
llm = get_local_llm("llama3.2:latest")
# Define Prompt Template 
from langchain_core.prompts import ChatPromptTemplate
template = """
Your are an assistant for question-answering tasks.
Use the following pieces of retrieved context to answer the question.
If you don't know, just say you don't know.
keep the answer concise.

Question: {question}
Context: {context}
"""
prompt = ChatPromptTemplate.from_template(template)
# Construct Rag Chain
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser, SimpleJsonOutputParser
rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()} 
    | prompt 
    | llm
    | StrOutputParser()
)

In [ ]:
from deepeval.metrics import AnswerRelevancyMetric, ContextualRelevancyMetric
from deepeval.test_case import LLMTestCase
@observe(type="retriever", 
         name="Retriever",
         metrics=[ContextualRelevancyMetric()], 
         embedder="llama3.2:latest")
def retrieve_context(question):
    docs = retriever.invoke(question)
    print(f"Retrieved {len(docs)} documents.")
    retrieved_contexts = [doc.page_content for doc in docs if doc.page_content is not None]
    update_current_span(
        test_case=LLMTestCase(input=question, retrieval_context=retrieved_contexts)
    )
    return retrieved_contexts

In [ ]:
@observe(type="rag_application", 
         name="RAG Application", 
         metrics=[ContextualRelevancyMetric(), AnswerRelevancyMetric()])
def rag_application(question):
    actual_response = rag_chain.invoke(question)
    print("RAG Response:", actual_response)
    context_list = retrieve_context(question)
    print("Retrieved Context:", context_list)
    update_current_span(
        test_case=LLMTestCase(
            input=question,
            actual_output=actual_response,
            retrieval_context=context_list
        )
    )
    return actual_response

In [ ]:
# rag_application("What is MCP?")

#### Evaluation of RAG Application

In [ ]:
from deepeval.dataset import Golden
from deepeval.dataset import EvaluationDataset

goldens = [Golden(input="What is MCP")]
dataset = EvaluationDataset(goldens=goldens)
for golden in dataset.evals_iterator():
     rag_application(golden.input)